# SE446 – Week 2B: Hands-On HDFS Exploration

## Exploring the Hadoop Distributed File System on a Real Cluster

---

### 🎯 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand HDFS Architecture**: Master/Worker topology with NameNode and DataNodes
2. **Connect to a Production Cluster**: Access our live HDFS cluster securely
3. **Execute HDFS Commands**: Create directories, upload/download files, manage data
4. **Understand Replication**: See how data is replicated across DataNodes
5. **Explore Block Distribution**: Understand how large files are split and distributed
6. **Prepare for MapReduce**: Understand how HDFS enables distributed processing

---

### 📋 Prerequisites

- Basic Python knowledge
- Understanding of file systems
- Completed Week 2A notebook (Big Data concepts)

---

### 🖥️ Our Cluster

You have access to a **production HDFS cluster** deployed on DigitalOcean:

| Component | Details |
|-----------|----------|
| **Hadoop Version** | 3.4.1 |
| **Total Nodes** | 3 (1 Master + 2 Workers) |
| **Total Storage** | 95.66 GB |
| **Replication Factor** | 2 |
| **Web UI** | https://hdfs.aniskoubaa.org |

---

## 1. Environment Setup

### 1.1 Install Required Packages

We'll use `paramiko` for SSH connections and `python-dotenv` for secure credential management.

In [1]:
# Install required packages
!pip install paramiko python-dotenv requests pandas tabulate -q

print("✅ Packages installed successfully!")

✅ Packages installed successfully!


### 1.2 Import Libraries

In [2]:
import paramiko
import os
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

print("📚 Libraries imported successfully!")
print(f"📅 Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📚 Libraries imported successfully!
📅 Session started: 2026-01-27 18:50:52


### 1.3 Configure Credentials Securely

**⚠️ Security Best Practice**: Never hardcode passwords in notebooks!

We'll use environment variables loaded from a `.env` file.

#### Option A: Local Development
Create a `.env` file in the same directory with:
```
HDFS_MASTER_HOST=134.209.172.50
HDFS_SSH_USER=root
HDFS_SSH_PASSWORD=your_password_here
HDFS_HADOOP_USER=hadoop
```

#### Option B: Google Colab
Use Colab's Secrets feature (🔑 icon in the sidebar).

In [3]:
# Detect environment and load credentials
def load_credentials():
    """Load HDFS credentials from .env file or Colab secrets."""
    
    # Check if running in Google Colab
    try:
        from google.colab import userdata
        IN_COLAB = True
        print("🌐 Running in Google Colab")
    except ImportError:
        IN_COLAB = False
        print("💻 Running locally")
    
    if IN_COLAB:
        # Load from Colab secrets
        try:
            credentials = {
                'host': userdata.get('HDFS_MASTER_HOST'),
                'ssh_user': userdata.get('HDFS_SSH_USER'),
                'ssh_password': userdata.get('HDFS_SSH_PASSWORD'),
                'hadoop_user': userdata.get('HDFS_HADOOP_USER')
            }
            print("✅ Credentials loaded from Colab Secrets")
        except Exception as e:
            print(f"❌ Error loading Colab secrets: {e}")
            print("Please add secrets using the 🔑 icon in the sidebar")
            return None
    else:
        # Load from .env file
        from dotenv import load_dotenv
        
        env_path = Path('.env')
        if env_path.exists():
            print(f"📂 Found .env file at: {env_path.absolute()}")
            load_dotenv(env_path)
            credentials = {
                'host': os.getenv('HDFS_MASTER_HOST'),
                'ssh_user': os.getenv('HDFS_SSH_USER'),
                'ssh_password': os.getenv('HDFS_SSH_PASSWORD'),
                'hadoop_user': os.getenv('HDFS_HADOOP_USER', 'hadoop')
            }
            
            # Validate credentials
            missing = []
            for key, value in credentials.items():
                if key != 'hadoop_user' and (not value or value.strip() == ''):
                    missing.append(key)
            
            if missing:
                print(f"❌ Missing or empty credentials: {', '.join(missing)}")
                print("\n📋 Current values:")
                for key, value in credentials.items():
                    if key == 'ssh_password':
                        print(f"   {key}: {'*' * len(value) if value else 'NOT SET'}")
                    else:
                        print(f"   {key}: {value if value else 'NOT SET'}")
                print("\n💡 Check your .env file format. It should look like:")
                print("""
HDFS_MASTER_HOST=134.209.172.50
HDFS_SSH_USER=root
HDFS_SSH_PASSWORD=your_actual_password
HDFS_HADOOP_USER=hadoop
                """)
                print("\n⚠️ Common issues:")
                print("   • No spaces around = sign")
                print("   • No quotes around values")
                print("   • No comments on the same line")
                return None
            
            print("✅ Credentials loaded from .env file")
            print(f"   Host: {credentials['host']}")
            print(f"   SSH User: {credentials['ssh_user']}")
            print(f"   Password: {'*' * 8}")
            print(f"   Hadoop User: {credentials['hadoop_user']}")
        else:
            print("❌ No .env file found!")
            print(f"   Expected location: {env_path.absolute()}")
            print("\nPlease create a .env file with the following content:")
            print("""
HDFS_MASTER_HOST=134.209.172.50
HDFS_SSH_USER=root
HDFS_SSH_PASSWORD=your_password
HDFS_HADOOP_USER=hadoop
            """)
            return None
    
    return credentials

# Load credentials
CREDENTIALS = load_credentials()

💻 Running locally
📂 Found .env file at: /Applications/XAMPP/xamppfiles/htdocs/aniskoubaa.org/se446/big_data_course/lectures/week02/notebooks/.env
✅ Credentials loaded from .env file
   Host: 134.209.172.50
   SSH User: root
   Password: ********
   Hadoop User: hadoop


---

## 2. Understanding HDFS Architecture

Before connecting, let's understand what we're working with.

### 2.1 Cluster Topology Overview

```
                    ┌─────────────────────────────────────┐
                    │           STUDENTS                  │
                    │      (HTTPS via Browser)            │
                    └────────────────┬────────────────────┘
                                     │
                                     ▼
                    ┌─────────────────────────────────────┐
                    │      NGINX REVERSE PROXY            │
                    │   SSL/TLS + Authentication          │
                    │      hdfs.aniskoubaa.org            │
                    └────────────────┬────────────────────┘
                                     │
         ┌───────────────────────────┼───────────────────────────┐
         │                           │                           │
         ▼                           ▼                           ▼
┌─────────────────┐       ┌─────────────────┐       ┌─────────────────┐
│   MASTER NODE   │       │  WORKER NODE 1  │       │  WORKER NODE 2  │
│ 134.209.172.50  │◄─────►│ 104.131.191.109 │◄─────►│ 157.245.211.168 │
│                 │       │                 │       │                 │
│   • NameNode    │       │   • DataNode    │       │   • DataNode    │
│   • Secondary   │       │   • 48.28 GB    │       │   • 47.39 GB    │
│     NameNode    │       │                 │       │                 │
└─────────────────┘       └─────────────────┘       └─────────────────┘
         │                           │                           │
         └───────────────────────────┴───────────────────────────┘
                           Heartbeats & Block Reports
```

In [4]:
# Define our cluster configuration
CLUSTER_CONFIG = {
    'master': {
        'hostname': 'master-node',
        'ip': '134.209.172.50',
        'role': 'Master',
        'components': ['NameNode', 'SecondaryNameNode'],
        'ports': {
            'namenode_rpc': 9000,
            'namenode_http': 9870,
            'secondary_http': 9868
        }
    },
    'workers': [
        {
            'hostname': 'worker-node-1',
            'ip': '104.131.191.109',
            'role': 'Worker',
            'components': ['DataNode'],
            'capacity_gb': 48.28
        },
        {
            'hostname': 'worker-node-2',
            'ip': '157.245.211.168',
            'role': 'Worker',
            'components': ['DataNode'],
            'capacity_gb': 47.39
        }
    ],
    'hdfs': {
        'replication_factor': 2,
        'block_size_mb': 128,
        'total_capacity_gb': 95.67
    }
}

In [5]:
# Display cluster info
print("🖥️  HDFS CLUSTER CONFIGURATION")
print("=" * 50)
print(f"\n📊 Master Node: {CLUSTER_CONFIG['master']['hostname']}")
print(f"   IP: {CLUSTER_CONFIG['master']['ip']}")
print(f"   Components: {', '.join(CLUSTER_CONFIG['master']['components'])}")

print("\n📊 Worker Nodes:")
for i, worker in enumerate(CLUSTER_CONFIG['workers'], 1):
    print(f"   {i}. {worker['hostname']} ({worker['ip']})")
    print(f"      Capacity: {worker['capacity_gb']} GB")

print(f"\n📊 HDFS Settings:")
print(f"   Replication Factor: {CLUSTER_CONFIG['hdfs']['replication_factor']}")
print(f"   Block Size: {CLUSTER_CONFIG['hdfs']['block_size_mb']} MB")
print(f"   Total Capacity: {CLUSTER_CONFIG['hdfs']['total_capacity_gb']} GB")
print(f"   Effective Capacity: {CLUSTER_CONFIG['hdfs']['total_capacity_gb']/2:.2f} GB (due to replication)")

🖥️  HDFS CLUSTER CONFIGURATION

📊 Master Node: master-node
   IP: 134.209.172.50
   Components: NameNode, SecondaryNameNode

📊 Worker Nodes:
   1. worker-node-1 (104.131.191.109)
      Capacity: 48.28 GB
   2. worker-node-2 (157.245.211.168)
      Capacity: 47.39 GB

📊 HDFS Settings:
   Replication Factor: 2
   Block Size: 128 MB
   Total Capacity: 95.67 GB
   Effective Capacity: 47.84 GB (due to replication)


### 2.2 Key HDFS Concepts

| Concept | Description |
|---------|-------------|
| **NameNode** | Master server that manages file system namespace and regulates access |
| **DataNode** | Worker servers that store actual data blocks |
| **Block** | Fixed-size chunk of data (default 128 MB) |
| **Replication** | Multiple copies of each block for fault tolerance |
| **Heartbeat** | Periodic signal from DataNode to NameNode (every 3 seconds) |
| **Block Report** | List of all blocks on a DataNode, sent periodically |

---

## 3. Connecting to the HDFS Cluster

### 3.1 Create SSH Connection Helper

In [6]:
class HDFSClusterConnection:
    """
    A helper class to connect to our HDFS cluster via SSH
    and execute HDFS commands.
    """
    
    def __init__(self, credentials):
        self.host = credentials['host']
        self.ssh_user = credentials['ssh_user']
        self.ssh_password = credentials['ssh_password']
        self.hadoop_user = credentials.get('hadoop_user', 'hadoop')
        self.client = None
        
    def connect(self):
        """Establish SSH connection to the master node."""
        try:
            self.client = paramiko.SSHClient()
            self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            self.client.connect(
                hostname=self.host,
                username=self.ssh_user,
                password=self.ssh_password,
                timeout=30
            )
            print(f"✅ Connected to master node at {self.host}")
            return True
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False
    
    def disconnect(self):
        """Close SSH connection."""
        if self.client:
            self.client.close()
            print("🔌 Disconnected from cluster")
    
    def execute_hdfs_command(self, command, show_output=True):
        """
        Execute an HDFS command as the hadoop user.
        
        Args:
            command: HDFS command (without 'hdfs dfs' prefix)
            show_output: Whether to print the output
            
        Returns:
            tuple: (stdout, stderr, exit_code)
        """
        if not self.client:
            print("❌ Not connected. Call connect() first.")
            return None, None, -1
        
        # Build the full command
        full_command = f"sudo -u {self.hadoop_user} hdfs dfs {command}"
        
        if show_output:
            print(f"\n🔧 Executing: hdfs dfs {command}")
            print("-" * 50)
        
        stdin, stdout, stderr = self.client.exec_command(full_command)
        
        out = stdout.read().decode('utf-8')
        err = stderr.read().decode('utf-8')
        exit_code = stdout.channel.recv_exit_status()
        
        if show_output:
            if out:
                print(out)
            if err and exit_code != 0:
                print(f"⚠️ Error: {err}")
        
        return out, err, exit_code
    
    def execute_admin_command(self, command, show_output=True):
        """
        Execute an HDFS admin command.
        
        Args:
            command: Admin command (without 'hdfs dfsadmin' prefix)
        """
        if not self.client:
            print("❌ Not connected. Call connect() first.")
            return None, None, -1
        
        full_command = f"sudo -u {self.hadoop_user} hdfs dfsadmin {command}"
        
        if show_output:
            print(f"\n🔧 Executing: hdfs dfsadmin {command}")
            print("-" * 50)
        
        stdin, stdout, stderr = self.client.exec_command(full_command)
        
        out = stdout.read().decode('utf-8')
        err = stderr.read().decode('utf-8')
        exit_code = stdout.channel.recv_exit_status()
        
        if show_output:
            if out:
                print(out)
            if err and exit_code != 0:
                print(f"⚠️ Error: {err}")
        
        return out, err, exit_code
    
    def execute_shell_command(self, command, show_output=True):
        """Execute a general shell command."""
        if not self.client:
            print("❌ Not connected. Call connect() first.")
            return None, None, -1
        
        if show_output:
            print(f"\n🔧 Executing: {command}")
            print("-" * 50)
        
        stdin, stdout, stderr = self.client.exec_command(command)
        
        out = stdout.read().decode('utf-8')
        err = stderr.read().decode('utf-8')
        exit_code = stdout.channel.recv_exit_status()
        
        if show_output:
            if out:
                print(out)
            if err and exit_code != 0:
                print(f"⚠️ Error: {err}")
        
        return out, err, exit_code

print("✅ HDFSClusterConnection class defined")

✅ HDFSClusterConnection class defined


### 3.2 Establish Connection

In [7]:
# Create connection instance
if CREDENTIALS:
    hdfs = HDFSClusterConnection(CREDENTIALS)
    
    # Connect to the cluster
    if hdfs.connect():
        print("\n🎉 You are now connected to the HDFS cluster!")
        print("📍 Master Node: master-node (134.209.172.50)")
    else:
        print("\n⚠️ Please check your credentials and try again.")
else:
    print("⚠️ Credentials not loaded. Please configure .env file or Colab secrets.")

✅ Connected to master node at 134.209.172.50

🎉 You are now connected to the HDFS cluster!
📍 Master Node: master-node (134.209.172.50)


---

## 4. Exploring the NameNode (Master)

The **NameNode** is the brain of HDFS. It:
- Manages the file system namespace (directory structure)
- Stores metadata (file names, permissions, block locations)
- Does NOT store actual data

### 4.1 Check Cluster Health

In [8]:
# Get cluster report from NameNode
print("📊 HDFS CLUSTER REPORT")
print("=" * 60)
hdfs.execute_admin_command("-report")

📊 HDFS CLUSTER REPORT

🔧 Executing: hdfs dfsadmin -report
--------------------------------------------------
⚠️ Error: sudo: hdfs: command not found



('', 'sudo: hdfs: command not found\n', 1)

### 🎯 Exercise 4.1: Understanding the Report

Look at the output above and answer:

1. How many **Live datanodes** are there?
2. What is the **Configured Capacity**?
3. What is the **DFS Used** percentage?
4. What is the **Block Pool Used**?

In [9]:
# Your answers here (uncomment and fill in):

# answer_4_1_1 = "___"  # Number of live datanodes
# answer_4_1_2 = "___"  # Configured capacity
# answer_4_1_3 = "___"  # DFS Used percentage
# answer_4_1_4 = "___"  # Block pool used

### 4.2 Check NameNode Safe Mode Status

In [10]:
# Check if NameNode is in Safe Mode
print("🔒 SAFE MODE STATUS")
print("=" * 40)
hdfs.execute_admin_command("-safemode get")

print("\n💡 Safe Mode: When NameNode is in safe mode, it is read-only.")
print("   This happens during startup or maintenance.")

🔒 SAFE MODE STATUS

🔧 Executing: hdfs dfsadmin -safemode get
--------------------------------------------------
⚠️ Error: sudo: hdfs: command not found


💡 Safe Mode: When NameNode is in safe mode, it is read-only.
   This happens during startup or maintenance.


### 4.3 View NameNode Configuration

In [11]:
# Check key HDFS configuration
print("⚙️ KEY HDFS CONFIGURATION")
print("=" * 50)

# Get replication factor
out, _, _ = hdfs.execute_shell_command(
    "grep -A1 'dfs.replication' /opt/hadoop/etc/hadoop/hdfs-site.xml",
    show_output=False
)
print(f"📋 Replication Factor Configuration:")
print(out)

# Get block size
out, _, _ = hdfs.execute_shell_command(
    "grep -A1 'dfs.blocksize' /opt/hadoop/etc/hadoop/hdfs-site.xml",
    show_output=False
)
print(f"📋 Block Size Configuration:")
print(out if out else "Using default: 128 MB")

⚙️ KEY HDFS CONFIGURATION
📋 Replication Factor Configuration:
    <name>dfs.replication</name>
    <value>2</value>

📋 Block Size Configuration:
Using default: 128 MB


---

## 5. Exploring DataNodes (Workers)

**DataNodes** are the workhorses of HDFS. They:
- Store actual data blocks
- Send heartbeats to NameNode every 3 seconds
- Report block information periodically
- Perform read/write operations

### 5.1 List All DataNodes

In [12]:
# Get DataNode information
print("📊 DATANODE INFORMATION")
print("=" * 60)

out, _, _ = hdfs.execute_admin_command("-report", show_output=False)

# Parse DataNode information
import re

datanode_pattern = r'Name: (\S+).*?Hostname: (\S+).*?Configured Capacity: (\d+).*?DFS Used: (\d+).*?DFS Remaining: (\d+)'
datanodes = re.findall(datanode_pattern, out, re.DOTALL)

if datanodes:
    print("\n📍 Live DataNodes:\n")
    for i, dn in enumerate(datanodes, 1):
        name, hostname, capacity, used, remaining = dn
        capacity_gb = int(capacity) / (1024**3)
        used_gb = int(used) / (1024**3)
        remaining_gb = int(remaining) / (1024**3)
        
        print(f"  DataNode {i}: {hostname}")
        print(f"    ├─ IP:Port: {name}")
        print(f"    ├─ Capacity: {capacity_gb:.2f} GB")
        print(f"    ├─ Used: {used_gb:.2f} GB ({(used_gb/capacity_gb)*100:.1f}%)")
        print(f"    └─ Remaining: {remaining_gb:.2f} GB")
        print()

📊 DATANODE INFORMATION


### 5.2 Understanding Heartbeats

DataNodes send **heartbeats** to the NameNode every 3 seconds to indicate they are alive.

If the NameNode doesn't receive a heartbeat for **10 minutes** (default), it marks the DataNode as **dead**.

In [13]:
# Check for dead nodes
print("💓 DATANODE HEALTH CHECK")
print("=" * 50)

# Look for "Dead datanodes" in the report
if "Dead datanodes (0)" in out or "Dead datanodes: 0" in out.replace('(', ':').replace(')', ''):
    print("\n✅ All DataNodes are healthy!")
    print("   └─ Heartbeats are being received from all nodes")
else:
    print("\n⚠️ Some DataNodes may be offline")
    print("   └─ Check the cluster report for details")

print("\n📝 Heartbeat Configuration:")
print("   ├─ Heartbeat Interval: 3 seconds")
print("   └─ Dead Node Timeout: 10 minutes")

💓 DATANODE HEALTH CHECK

⚠️ Some DataNodes may be offline
   └─ Check the cluster report for details

📝 Heartbeat Configuration:
   ├─ Heartbeat Interval: 3 seconds
   └─ Dead Node Timeout: 10 minutes


---

## 6. Basic HDFS Commands

HDFS commands are similar to Unix file system commands!

| HDFS Command | Unix Equivalent | Description |
|--------------|-----------------|-------------|
| `hdfs dfs -ls` | `ls` | List files |
| `hdfs dfs -mkdir` | `mkdir` | Create directory |
| `hdfs dfs -put` | `cp` (to HDFS) | Upload file |
| `hdfs dfs -get` | `cp` (from HDFS) | Download file |
| `hdfs dfs -cat` | `cat` | View file content |
| `hdfs dfs -rm` | `rm` | Delete file |
| `hdfs dfs -du` | `du` | Disk usage |

### 6.1 List Root Directory

In [14]:
# List the root directory
print("📁 HDFS ROOT DIRECTORY")
print("=" * 50)
hdfs.execute_hdfs_command("-ls /")

📁 HDFS ROOT DIRECTORY

🔧 Executing: hdfs dfs -ls /
--------------------------------------------------
⚠️ Error: sudo: hdfs: command not found



('', 'sudo: hdfs: command not found\n', 1)

### 6.2 Create Your Personal Directory

In [ ]:
# Create a directory for your experiments
# Replace 'student_demo' with your actual name/ID
STUDENT_NAME = "student_demo"  # ← CHANGE THIS!

student_dir = f"/students/{STUDENT_NAME}"

print(f"📁 Creating directory: {student_dir}")
hdfs.execute_hdfs_command(f"-mkdir -p {student_dir}")

# Verify creation
print(f"\n📋 Verifying directory exists:")
hdfs.execute_hdfs_command("-ls /students")

### 6.3 Upload a File to HDFS

In [ ]:
# First, create a sample file locally on the master node
sample_data = """id,name,department,salary
1,Alice,Engineering,75000
2,Bob,Marketing,65000
3,Charlie,Engineering,80000
4,Diana,HR,60000
5,Eve,Marketing,70000
"""

# Create the file on the remote server
create_file_cmd = f"echo '{sample_data}' > /tmp/employees.csv"
hdfs.execute_shell_command(create_file_cmd, show_output=False)
print("✅ Created sample file: /tmp/employees.csv")

# Upload to HDFS
print(f"\n📤 Uploading file to HDFS...")
hdfs.execute_hdfs_command(f"-put /tmp/employees.csv {student_dir}/employees.csv")

# Verify upload
print(f"\n📋 Files in {student_dir}:")
hdfs.execute_hdfs_command(f"-ls {student_dir}")

### 6.4 View File Contents

In [ ]:
# View file content using -cat
print("📄 FILE CONTENTS")
print("=" * 40)
hdfs.execute_hdfs_command(f"-cat {student_dir}/employees.csv")

### 6.5 Check File Statistics

In [ ]:
# Get detailed file information
print("📊 FILE STATISTICS")
print("=" * 50)

# File size
print("\n📏 File Size:")
hdfs.execute_hdfs_command(f"-du -h {student_dir}/employees.csv")

# Detailed stats
print("\n📋 Detailed Statistics:")
hdfs.execute_hdfs_command(f"-stat 'Replication: %r, Block Size: %o, Size: %b bytes' {student_dir}/employees.csv")

---

## 7. Understanding Replication

**Replication** is HDFS's primary mechanism for fault tolerance.

### 7.1 How Replication Works

```
                    ┌─────────────────┐
                    │    NameNode     │
                    │                 │
                    │  Block Map:     │
                    │  Block1 → DN1,DN2│
                    └────────┬────────┘
                             │
         ┌───────────────────┼───────────────────┐
         │                   │                   │
         ▼                   ▼                   ▼
┌─────────────────┐ ┌─────────────────┐ ┌─────────────────┐
│   DataNode 1    │ │   DataNode 2    │ │   DataNode 3    │
│                 │ │                 │ │                 │
│   ┌─────────┐   │ │   ┌─────────┐   │ │                 │
│   │ Block 1 │   │ │   │ Block 1 │   │ │   (no replica)  │
│   │ (copy)  │   │ │   │ (copy)  │   │ │                 │
│   └─────────┘   │ │   └─────────┘   │ │                 │
└─────────────────┘ └─────────────────┘ └─────────────────┘

With Replication Factor = 2, each block is stored on 2 different DataNodes
```

In [ ]:
# Check replication of our file
print("🔄 REPLICATION ANALYSIS")
print("=" * 50)

# Get file info including block locations
out, _, _ = hdfs.execute_shell_command(
    f"sudo -u hadoop hdfs fsck {student_dir}/employees.csv -files -blocks -locations",
    show_output=True
)

### 7.2 Change Replication Factor

In [ ]:
# Check current replication
print("📊 Current Replication Factor:")
hdfs.execute_hdfs_command(f"-stat '%r' {student_dir}/employees.csv")

# Note: We have only 2 DataNodes, so max replication is 2
print("\n💡 Our cluster has 2 DataNodes, so maximum replication = 2")
print("   If you try to set replication > 2, HDFS will only use 2")

### 🎯 Exercise 7.1: Understanding Replication Impact

If you upload a **100 MB** file with **replication factor = 2**:

1. How much **actual storage** is used on the cluster?
2. How many **copies** of each block exist?
3. What happens if **one DataNode fails**?

In [ ]:
# Your answers here:

# answer_7_1_1 = "___"  # Actual storage used
# answer_7_1_2 = "___"  # Number of copies
# answer_7_1_3 = "___"  # What happens on failure

---

## 8. Understanding Block Distribution

Large files are split into **blocks** (default 128 MB) and distributed across DataNodes.

### 8.1 How Block Distribution Works

```
Original File: 300 MB
Block Size: 128 MB

┌─────────────────────────────────────────────────────────────┐
│                     300 MB File                             │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼ Split into blocks
┌───────────────┐ ┌───────────────┐ ┌───────────────┐
│   Block 1     │ │   Block 2     │ │   Block 3     │
│   128 MB      │ │   128 MB      │ │    44 MB      │
└───────┬───────┘ └───────┬───────┘ └───────┬───────┘
        │                 │                 │
        ▼                 ▼                 ▼ Distribute across nodes
┌───────────────┐ ┌───────────────┐ ┌───────────────┐
│  DataNode 1   │ │  DataNode 2   │ │  DataNode 1   │
│   Block 1     │ │   Block 2     │ │   Block 3     │
│   Block 2     │ │   Block 1     │ │   Block 2     │
│   (replica)   │ │   (replica)   │ │   (replica)   │
└───────────────┘ └───────────────┘ └───────────────┘
```

In [ ]:
# Create a larger file to demonstrate block distribution
print("📦 CREATING LARGER FILE FOR BLOCK DEMONSTRATION")
print("=" * 50)

# Generate a 5MB file (enough to see distribution, small enough to be fast)
hdfs.execute_shell_command(
    "dd if=/dev/urandom of=/tmp/sample_5mb.bin bs=1M count=5 2>/dev/null",
    show_output=False
)
print("✅ Created 5 MB binary file")

# Upload to HDFS
print("\n📤 Uploading to HDFS...")
hdfs.execute_hdfs_command(f"-put /tmp/sample_5mb.bin {student_dir}/sample_5mb.bin")

# Check block distribution
print("\n📊 Block Distribution Analysis:")
hdfs.execute_shell_command(
    f"sudo -u hadoop hdfs fsck {student_dir}/sample_5mb.bin -files -blocks -locations"
)

### 8.2 Visualize Storage Distribution

In [ ]:
# Check storage usage on each DataNode
print("📊 STORAGE DISTRIBUTION ACROSS DATANODES")
print("=" * 50)

out, _, _ = hdfs.execute_admin_command("-report", show_output=False)

# Parse and display storage info
lines = out.split('\n')
current_node = None
storage_data = []

for line in lines:
    if 'Hostname:' in line:
        current_node = line.split(':')[1].strip()
    elif 'DFS Used:' in line and current_node:
        try:
            used = int(line.split(':')[1].strip().split()[0])
            used_gb = used / (1024**3)
            storage_data.append({'Node': current_node, 'Used (GB)': f"{used_gb:.4f}"})
        except:
            pass

if storage_data:
    df = pd.DataFrame(storage_data)
    print(df.to_string(index=False))
    print("\n💡 Notice how data is distributed across both DataNodes!")

---

## 9. Reading Files from Distributed Storage

When you read a file from HDFS, the client:
1. Asks NameNode for block locations
2. Reads blocks directly from DataNodes
3. Assembles blocks into the complete file

### 9.1 The Read Process

In [ ]:
# Demonstrate reading a file
print("📖 HDFS READ OPERATION")
print("=" * 50)

print("\n📋 Step 1: Get block locations from NameNode")
print("-" * 40)
hdfs.execute_shell_command(
    f"sudo -u hadoop hdfs fsck {student_dir}/employees.csv -files -blocks -locations 2>/dev/null | grep -E 'blk_|DatanodeInfoWithStorage'",
    show_output=True
)

print("\n📋 Step 2: Read file content (assembled from blocks)")
print("-" * 40)
hdfs.execute_hdfs_command(f"-cat {student_dir}/employees.csv")

### 9.2 Download File from HDFS

In [ ]:
# Download file from HDFS to local
print("📥 DOWNLOADING FILE FROM HDFS")
print("=" * 50)

# Download using -get
hdfs.execute_hdfs_command(f"-get {student_dir}/employees.csv /tmp/downloaded_employees.csv")

# Verify download
print("\n✅ File downloaded! Verifying content:")
hdfs.execute_shell_command("cat /tmp/downloaded_employees.csv")

---

## 10. Preparing for MapReduce

Now that you understand HDFS, you're ready for **MapReduce**!

### 10.1 Why HDFS + MapReduce Work Together

```
┌─────────────────────────────────────────────────────────────────────┐
│                        MapReduce Job                                 │
│                                                                      │
│   Input (HDFS)          Map Phase           Reduce Phase   Output   │
│                                                            (HDFS)   │
│   ┌─────────┐          ┌────────┐          ┌────────┐              │
│   │ Block 1 │  ───►    │ Map 1  │  ──┐     │        │              │
│   │(Node 1) │          └────────┘    │     │        │   ┌───────┐  │
│   └─────────┘                        ├───► │Reduce 1│──►│Result │  │
│   ┌─────────┐          ┌────────┐    │     │        │   └───────┘  │
│   │ Block 2 │  ───►    │ Map 2  │  ──┘     │        │              │
│   │(Node 2) │          └────────┘          └────────┘              │
│   └─────────┘                                                       │
│                                                                      │
│   Key Insight: Map tasks run WHERE the data is stored (data locality)│
└─────────────────────────────────────────────────────────────────────┘
```

**Key Concepts:**
- **Data Locality**: MapReduce moves computation to data, not data to computation
- **Parallel Processing**: Each block can be processed by a separate Map task
- **Fault Tolerance**: If a task fails, HDFS replication allows restart on another node

In [ ]:
# Check if MapReduce/YARN is available
print("🔍 CHECKING MAPREDUCE READINESS")
print("=" * 50)

# Check Hadoop version
print("\n📋 Hadoop Version:")
hdfs.execute_shell_command("hadoop version | head -3")

# Check HDFS status for MapReduce
print("\n📋 HDFS Status (for MapReduce input):")
hdfs.execute_hdfs_command(f"-count {student_dir}")

print("\n✅ Your HDFS cluster is ready for MapReduce!")
print("   └─ Next lecture: We'll run MapReduce jobs on this data")

### 10.2 Create Sample Data for MapReduce Lab

In [ ]:
# Create word count sample data (classic MapReduce example)
print("📝 CREATING SAMPLE DATA FOR MAPREDUCE LAB")
print("=" * 50)

word_count_data = """Hello World
Hello Hadoop
Hello HDFS
Hadoop is great
HDFS stores data
MapReduce processes data
Hello Big Data
Big Data is the future
Hadoop HDFS MapReduce
Data Data Data
"""

# Create and upload
hdfs.execute_shell_command(f"echo '{word_count_data}' > /tmp/wordcount_input.txt", show_output=False)
hdfs.execute_hdfs_command(f"-mkdir -p {student_dir}/mapreduce_lab")
hdfs.execute_hdfs_command(f"-put /tmp/wordcount_input.txt {student_dir}/mapreduce_lab/input.txt")

print("\n✅ Sample data created for MapReduce lab!")
print(f"\n📋 File location: {student_dir}/mapreduce_lab/input.txt")
print("\n📄 Content:")
hdfs.execute_hdfs_command(f"-cat {student_dir}/mapreduce_lab/input.txt")

---

## 11. HDFS Command Reference

### Quick Reference Card

In [ ]:
# Print HDFS command reference
print("📚 HDFS COMMAND QUICK REFERENCE")
print("=" * 70)

commands = [
    ["Category", "Command", "Description"],
    ["─" * 10, "─" * 30, "─" * 25],
    ["Navigation", "hdfs dfs -ls <path>", "List directory contents"],
    ["", "hdfs dfs -ls -R <path>", "List recursively"],
    ["", "hdfs dfs -ls -h <path>", "Human-readable sizes"],
    ["─" * 10, "─" * 30, "─" * 25],
    ["Directory", "hdfs dfs -mkdir <path>", "Create directory"],
    ["", "hdfs dfs -mkdir -p <path>", "Create with parents"],
    ["─" * 10, "─" * 30, "─" * 25],
    ["File Ops", "hdfs dfs -put <src> <dst>", "Upload file"],
    ["", "hdfs dfs -get <src> <dst>", "Download file"],
    ["", "hdfs dfs -cat <file>", "View file content"],
    ["", "hdfs dfs -tail <file>", "View last 1KB"],
    ["", "hdfs dfs -head <file>", "View first 1KB"],
    ["─" * 10, "─" * 30, "─" * 25],
    ["Delete", "hdfs dfs -rm <file>", "Delete file"],
    ["", "hdfs dfs -rm -r <dir>", "Delete directory"],
    ["", "hdfs dfs -rm -skipTrash <f>", "Delete permanently"],
    ["─" * 10, "─" * 30, "─" * 25],
    ["Info", "hdfs dfs -du -h <path>", "Disk usage"],
    ["", "hdfs dfs -df -h", "Free space"],
    ["", "hdfs dfs -stat <file>", "File statistics"],
    ["", "hdfs dfs -count <path>", "Count files/dirs/bytes"],
    ["─" * 10, "─" * 30, "─" * 25],
    ["Admin", "hdfs dfsadmin -report", "Cluster report"],
    ["", "hdfs fsck <path>", "File system check"],
]

for row in commands:
    print(f"{row[0]:<12} {row[1]:<32} {row[2]}")

---

## 12. Cleanup and Disconnect

In [ ]:
# Optional: Clean up test files
print("🧹 CLEANUP (Optional)")
print("=" * 50)

# Uncomment to delete test files:
# hdfs.execute_hdfs_command(f"-rm -r {student_dir}")
# print(f"✅ Deleted {student_dir}")

print("\n📋 Your files in HDFS:")
hdfs.execute_hdfs_command(f"-ls -R {student_dir}")

In [ ]:
# Disconnect from cluster
hdfs.disconnect()
print("\n✅ Session complete!")

---

## 📝 Summary

In this notebook, you learned:

| Topic | Key Takeaway |
|-------|---------------|
| **HDFS Architecture** | Master (NameNode) + Workers (DataNodes) |
| **NameNode** | Manages metadata, doesn't store data |
| **DataNode** | Stores actual blocks, sends heartbeats |
| **Blocks** | Files are split into 128 MB chunks |
| **Replication** | Each block stored on multiple nodes |
| **Commands** | Similar to Unix: ls, mkdir, put, get, cat |
| **MapReduce Ready** | HDFS provides input for distributed processing |

---

## 🎯 Next Steps

1. **Explore More**: Use the Web UI at https://hdfs.aniskoubaa.org
2. **Practice**: Upload larger files and observe block distribution
3. **Prepare**: Review MapReduce concepts for the next lecture

---

## 📚 Resources

- [Apache Hadoop Documentation](https://hadoop.apache.org/docs/r3.4.1/)
- [HDFS Architecture Guide](https://hadoop.apache.org/docs/r3.4.1/hadoop-project-dist/hadoop-hdfs/HdfsDesign.html)
- Cluster Web UI: https://hdfs.aniskoubaa.org (Username: admin)